In [8]:
# spread sheets関連のimport
import gspread_dataframe as gd
import gspread
import json
from oauth2client.service_account import ServiceAccountCredentials

In [9]:
#2つのAPIを記述しないとリフレッシュトークンを3600秒毎に発行し続けなければならない
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(r"C:\Users\ballr\Downloads\spreadingspreadsheets-f10083e9db88.json", scope)
#OAuth2の資格情報を使用してGoogle APIにログインします。
gc = gspread.authorize(credentials)
# Connecting with `gspread` here
ws = gc.open("company_url").worksheet("シート1")
my_df = gd.get_as_dataframe(ws)
s = my_df['url'].dropna()
url_list = s.values.tolist()  

In [10]:
# scraping関連のimport
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from tqdm import tqdm_notebook as tqdm
import time
from IPython.core.display import display

In [11]:
# データのスクレイピング
def scraping(url_list):
    for url in tqdm(url_list):
        try:
            html_doc = requests.get(url,verify=r"C:\Users\ballr\Downloads\certs.pem")   #  contentを渡すと文字化けが減らせる。
        except:
            pass
        soup = BeautifulSoup(html_doc.content,'html.parser')
        r = soup.find('body')  # file代わりにpandasを使っている。
        df = pd.Series(r.text) # textから改行の除外
        word_list = df[0].split("\n") # 改行で空いた分をさらに抜く
        Sentence_list = [word for word in word_list if word != ""]
        Morphological(Sentence_list)
        time.sleep(3)

In [12]:
# 形態素解析関連のimport
import sys
import MeCab
from collections import Counter

In [13]:
list = []
keyword_list = ["情報","事業","IR","採用","グループ","企業","ニュース","会社","サービス","CSR"]
information = []
business = []
IR = []
Adopt = []
group = []
kigyou = []
news = []
kaisha = []
service = []
CSR = []
LIST_LIST = [information, business, IR, Adopt, group, kigyou, news, kaisha, service, CSR]

# 形態素分析をリスト化する関数
def list_dictionary(m_article):
    for row in m_article.split("\n"):
        tag = row.split('\t')[0]
        if tag =="EOS":
            break
        else:
            pos = row.split("\t")[1]
            slice = pos[:2]
            d = {slice : tag}
            list.append(d)
        

# 形態素分析を実行する関数
def Morphological(Sentence_list):
    for sentence in Sentence_list:
        m = MeCab.Tagger ()
        m_article = m.parse (sentence)
        list_dictionary(m_article)
    
    counter(list)

# 名詞の数をカウントする関数
def counter(list):
    noun_list = [dic["名詞"] for dic in list if sorted(dic.keys())[0] == "名詞" and len(dic["名詞"]) > 1]
    counter = Counter(noun_list)
    for keyword in keyword_list:
        for List in LIST_LIST:
            for k,v in counter.items():
                if k == keyword:
                    List.append(v)



df2 = pd.DataFrame.from_dict({"information": information, "business": business, "IR": IR, "Adopt": Adopt, 
                              "group": group, "kigyou": kigyou, "news": news, "kaisha": kaisha, "service": service, 
                              "CSR": CSR})
            
        
# まとめ
def main():
    scraping(url_list)

main()

In [14]:
df2 = pd.DataFrame.from_dict({"information": information, "business": business, "IR": IR, "Adopt": Adopt, 
                              "group": group, "kigyou": kigyou, "news": news, "kaisha": kaisha, "service": service, 
                              "CSR": CSR})

display(df2)
print(information)
print(business)

,information,business,IR,Adopt,group,kigyou,news,kaisha,service,CSR
0,2,2,2,2,2,2,2,2,2,2
1,1,1,1,1,1,1,1,1,1,1
2,8,8,8,8,8,8,8,8,8,8
3,2,2,2,2,2,2,2,2,2,2
4,2,2,2,2,2,2,2,2,2,2
5,2,2,2,2,2,2,2,2,2,2
6,2,2,2,2,2,2,2,2,2,2
7,1,1,1,1,1,1,1,1,1,1
8,45,45,45,45,45,45,45,45,45,45
9,3,3,3,3,3,3,3,3,3,3


[2, 1, 8, 2, 2, 2, 2, 1, 45, 3, 8, 13, 34, 9, 14, 4, 4, 20, 48, 3, 8, 13, 34, 9, 14, 6, 5, 20, 52, 4, 8, 14, 43, 9, 18, 9, 7, 22, 64, 7, 8, 14, 49, 11, 22, 15, 10, 22, 71, 11, 12, 14, 56, 16, 24, 18, 10, 25, 83, 21, 15, 22, 59, 22, 33, 19, 15, 34, 88, 21, 15, 22, 59, 26, 35, 19, 15, 34, 88, 21, 15, 22, 59, 26, 35, 19, 15, 34, 133, 33, 40, 35, 93, 30, 41, 36, 16, 34]
[2, 1, 8, 2, 2, 2, 2, 1, 45, 3, 8, 13, 34, 9, 14, 4, 4, 20, 48, 3, 8, 13, 34, 9, 14, 6, 5, 20, 52, 4, 8, 14, 43, 9, 18, 9, 7, 22, 64, 7, 8, 14, 49, 11, 22, 15, 10, 22, 71, 11, 12, 14, 56, 16, 24, 18, 10, 25, 83, 21, 15, 22, 59, 22, 33, 19, 15, 34, 88, 21, 15, 22, 59, 26, 35, 19, 15, 34, 88, 21, 15, 22, 59, 26, 35, 19, 15, 34, 133, 33, 40, 35, 93, 30, 41, 36, 16, 34]
